In [2]:
# 7th March 2023 :: @12:43am


In [3]:
import pandas as pd, numpy as np, torch

import matplotlib.pyplot as plt
import seaborn as sns
import warnings, wget

warnings.filterwarnings("ignore")
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input, Flatten, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from transformers import AutoTokenizer, TFDistilBertForSequenceClassification
import pandas as pd
import torch

"""Importing libraries"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("fivethirtyeight")

from tqdm import tqdm
import time
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.model_selection import (
    train_test_split,
    GridSearchCV,
    StratifiedKFold,
    KFold,
    cross_val_score,
)

import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
import xgboost as xgb
from sklearn import preprocessing, model_selection, pipeline
from sklearn.metrics import f1_score, roc_auc_score

from keras.models import Sequential
from keras.layers import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import Embedding
from keras.layers import BatchNormalization
from keras.utils import np_utils
from keras.layers import (
    GlobalMaxPooling1D,
    Conv1D,
    MaxPooling1D,
    Flatten,
    Bidirectional,
    SpatialDropout1D,
)
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from transformers import BertTokenizer
from collections.abc import Iterable
from torchtext import data
from transformers import BertModel
import torch.nn as nn
from transformers.optimization import *
import torch.optim as optim
import math
from tensorflow.keras.utils import plot_model
from wordcloud import WordCloud


from helper_prabowo_ml import (
    clean_html,
    remove_links,
    remove_special_characters,
    removeStopWords,
    remove_,
    remove_digits,
    lower,
    email_address,
    non_ascii,
    punct,
)


In [4]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("jjzha/jobspanbert-base-cased")
model = AutoModel.from_pretrained("jjzha/jobspanbert-base-cased")


Some weights of the model checkpoint at jjzha/jobspanbert-base-cased were not used when initializing BertModel: ['cls.span_predictions.query_start_transform.LayerNorm.weight', 'cls.span_predictions.start_transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.span_predictions.query_start_transform.dense.weight', 'cls.predictions.transform.dense.weight', 'cls.span_predictions.end_transform.LayerNorm.weight', 'cls.span_predictions.start_transform.dense.weight', 'cls.span_predictions.end_classifier', 'cls.span_predictions.start_transform.LayerNorm.bias', 'cls.span_predictions.query_end_transform.dense.bias', 'cls.span_predictions.query_end_transform.LayerNorm.bias', 'cls.span_predictions.end_transform.dense.bias', 'cls.span_predictions.query_start_transform.dense.bias', 'cls.span_predictions.query_end_transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.span_predictions.query_end_transform.LayerNorm.weight', 'cls.span_predictions.start_clas

In [5]:
df = pd.read_csv("datasets/linkedin-jobs-machine-learning-uk-2055_03062023.csv")


In [6]:
import ast


# Define function to extract key-value pairs from criteria column
def extract_criteria(criteria):
    criteria_dict = {}
    # Convert string to list of dictionaries using ast.literal_eval()
    criteria_list = ast.literal_eval(criteria)
    # Loop through each dictionary in the list
    for d in criteria_list:
        # Loop through each key-value pair in the dictionary
        for key, value in d.items():
            # Add key-value pair to criteria_dict
            criteria_dict[key] = value
    return criteria_dict


# Apply function to 'criteria' column
df_criteria = df["criteria"].apply(extract_criteria).apply(pd.Series, dtype=object)

# Concatenate criteria DataFrame with original DataFrame
df = pd.concat([df.drop("criteria", axis=1), df_criteria], axis=1)


df = df.rename(
    columns={
        "Seniority level": "seniority_level",
        "Employment type": "employment_type",
        "Job function": "job_function",
        "Industries": "industries",
    }
)
# Display the result
df.head(3)


,title,company,description,onsite_remote,salary,location,posted_date,link,seniority_level,employment_type,job_function,industries
0,Junior Data Scientist,Oscar Technology,Role - Junior Data ScientistLocation - Manches...,onsite,NaN,"Manchester, England, United Kingdom",2023-02-28,https://uk.linkedin.com/jobs/view/junior-data-...,Entry level,Full-time,Engineering and Information Technology,Staffing and Recruiting
1,Graduate Machine Learning / Data Scientist,Cambridge Consultants,IntroductionWe are looking for talented mathem...,onsite,NaN,"Cambridge, England, United Kingdom",2023-02-27,https://uk.linkedin.com/jobs/view/graduate-mac...,Entry level,Full-time,Engineering and Information Technology,Design Services
2,Machine Learning Engineer,orbit,🚀Machine Learning Engineer🎥 Digital Media Serv...,onsite,NaN,"Leeds, England, United Kingdom",2023-02-07,https://uk.linkedin.com/jobs/view/machine-lear...,Not Applicable,Full-time,Information Technology,"Technology, Information and Internet"


In [7]:
df = df[['description', 'title']]

In [8]:
# Define function to normalize job titles
def normalize_job_title(title):
    # Convert title to lowercase
    title = title.lower()
    # Remove punctuation
    title = (
        title.replace(",", "")
        .replace(".", "")
        .replace(":", "")
        .replace("(", "")
        .replace(")", "")
        .replace("-", " ")
        .replace("/", " ")
    )
    # Tokenize title into words
    words = word_tokenize(title)
    # Remove stop words
    stop_words = set(stopwords.words("english"))
    words = [word for word in words if word not in stop_words]
    # Extract keywords from words
    keywords = []
    for word in words:
        if word in ["scientist", "engineer", "manager", "analyst"]:
            keywords.append(word)
    # Join keywords into a string
    return " ".join(keywords)


# Apply function to 'titles' column where the job title contains "machine learning" or "data scientist"
df.loc[
    df["title"].str.contains("machine learning|data scientist|data analyst", case=False),
    "normalized_job_title",
] = df["title"].apply(normalize_job_title)

# Display the result
df


,description,title,normalized_job_title
0,Role - Junior Data ScientistLocation - Manches...,Junior Data Scientist,scientist
1,IntroductionWe are looking for talented mathem...,Graduate Machine Learning / Data Scientist,scientist
2,🚀Machine Learning Engineer🎥 Digital Media Serv...,Machine Learning Engineer,engineer
3,"""Please note that we are currently unable to p...",Deep Learning Engineer,NaN
4,The OpportunityOur mission is to change the wa...,Data Scientist,scientist
...,...,...,...
1261,"Kantar is the world’s leading data, insights a...",Data Analyst,analyst
1262,Who are we?CellVoyant is a biotechnology compa...,Machine Learning Engineer,engineer
1263,The Opportunity Our mission is to change the w...,Data Scientist,scientist
1264,Role: Data and Insights AnalystLocation: Old T...,Data and Insights Analyst,NaN


In [9]:
labels_dict = {}

for idx, label in enumerate(df.title.unique()):
    labels_dict[label] = idx

labels_dict


{'Junior Data Scientist': 0,
 'Graduate Machine Learning / Data Scientist': 1,
 'Machine Learning Engineer': 2,
 'Deep Learning Engineer': 3,
 'Data Scientist': 4,
 'Applied Machine Learning Engineer': 5,
 'Graduate Machine Learning Software Engineer': 6,
 'Artificial Intelligence / Machine Learning Engineer - Autonomous': 7,
 'Applied Data Scientist': 8,
 'Data Engineer (Machine Learning)': 9,
 'Data Analyst': 10,
 'Junior/Senior/Chief AI Scientist': 11,
 'Data Analyst - Marketing': 12,
 'Senior Machine Learning Engineer': 13,
 'Data Scientist Lead': 14,
 'Machine Learning Scientist': 15,
 'Machine Learning Engineer (Internship/Part-Time)': 16,
 'Computer Vision Research Engineer - Deep Learning': 17,
 'ML Engineer Intern': 18,
 'Senior Data Scientist': 19,
 'Lead Data Scientist': 20,
 'AI Data Scientist - Contract or Perm': 21,
 'Machine Learning Engineer, Modeling': 22,
 'Data Science Manager': 23,
 'Scientific Algorithm Developer - Machine Learning': 24,
 'Software / Machine Learni

In [10]:
df.title = df.title.apply(func=lambda x: labels_dict[x])
df.title = df.title.astype(np.int64)


In [11]:
df = df[["description", "title"]]


In [12]:
def text_preprocess(data, col):
    data[col] = data[col].apply(func=clean_html)
    data[col] = data[col].apply(func=remove_)
    data[col] = data[col].apply(func=removeStopWords)
    data[col] = data[col].apply(func=remove_digits)
    data[col] = data[col].apply(func=remove_links)
    data[col] = data[col].apply(func=remove_special_characters)
    data[col] = data[col].apply(func=punct)
    data[col] = data[col].apply(func=non_ascii)
    data[col] = data[col].apply(func=email_address)
    data[col] = data[col].apply(func=lower)
    return data


In [13]:
preprocessed_df = text_preprocess(df, "description")

preprocessed_df.head()


,description,title
0,role junior data scientistlocation manchester ...,0
1,introductionwe looking talented mathematicians...,1
2,machine learning engineer digital media servic...,2
3,please note currently unable provide visa spon...,3
4,the opportunityour mission change way world wo...,4


In [14]:
preprocessed_df["description"].iloc[0][:100]


'role junior data scientistlocation manchester can remoteskills python essential full training provid'

In [15]:
preprocessed_df
df = preprocessed_df


In [16]:
%%time
# Tokenization
tokenized = df['description'].apply((lambda x: tokenizer.encode(x, max_length = 48, add_special_tokens=True, truncation=True)))
tokenized.shape

CPU times: user 697 ms, sys: 2.19 ms, total: 699 ms
Wall time: 697 ms


(1266,)

In [17]:
target = df["title"]


In [18]:
# Padding ==> convert 1D array to 2D array
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0] * (max_len - len(i)) for i in tokenized.values])
np.array(padded).shape


(1266, 48)

In [19]:
# Masking ==>  ignore (mask) the padding we've added
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape


(1266, 48)

In [20]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [21]:
features = last_hidden_states[0][:, 0, :].numpy()
labels = df["title"]


In [22]:
train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels
)


In [23]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)


LogisticRegression()

In [24]:
lr_clf.score(test_features, test_labels)


0.580441640378549

In [25]:
from sklearn.dummy import DummyClassifier

clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


Dummy classifier score: 0.290 (+/- 0.00)
